In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

# 各種数値

In [12]:
# 順に、軸荷重、径荷重、軸振動数、径振動数
rocket = np.array([6.0, 2.5, 30.0, 10.0])
reentry = np.array([0.0, 0.0, 0.0, 0.0])

#要求を決定
required = np.max([rocket,reentry],axis=0)

In [14]:
torayca_t800h = {'density':1.6, #g/cm^3
                 't　ension':2850, #MPa
                 'comp':1700, #MPa
                 'E':155, #GPa
                 'poisson':0.3  
                }

kevlar49 = {'density':1.45, #g/cm^3
            't　ension':3000, #MPa
            'comp':300, #MPa
            'E':112.4, #GPa
            'poisson':0.3  
            }

al7075 = {'density':2.8, #g/cm^3
          't　ension':550, #MPa
          'comp':500, #MPa
          'E':70, #GPa
          'shear':330, #MPa
          'poisson':0.3  
         }

In [30]:
def get_k_c(material,L,r,t):
    Z_L = L**2/r/t * np.sqrt(1-material['poisson']**2)
    k_c = (10**0.6)*Z_L
    return k_c

In [33]:
def get_sigma_buckling_cylinder(material,L,r,t):
    k_c = get_k_c(material,L,r,t)
    E = material['E']
    nu = material['poisson']
    return k_c*np.pi**2*E/12/((1-nu)**2)*(t/L)**2

In [34]:
def get_sigma(required,L,r,t,mass):
    """
    最終的にこのsigmaがsigma_buckling_cylinderを下回っていることを確認したい
    """
    F = required[0] * mass
    A = 2*np.pi*r*t
    return F/A

In [37]:
def get_f_axis(material,M,L,r,t):
    A = 2*np.pi*r*t
    E = material['E']
    f_axis = 0.250*np.sqrt(A*E/M/L)
    return f_axis

def get_f_rad(material,M,L,r,t):
    E = material['E']
    I = np.pi*(L**4-(L-t)**4)/64
    f_rad = 0.560*np.sqrt(E*I/M/L**3)
    return f_rad

In [39]:
def check():
    log = ['不合格','合格']
    
    sigma_buckling_cylinder = get_sigma_buckling_cylinder(material,L,r,t)
    sigma = get_sigma(required,L,r,t,mass)
    print('=================================')
    print('縦荷重合否:{0}'.format(log[sigma_buckling_cylinder>sigma]))
    print('縦荷重安全率:{0}\n'.format(sigma_buckling_cylinder/sigma))
    
    print('=================================')    
    f_axis = get_f_axis(material,M,L,r,t)
    f_axis_req = required[2]
    print('縦振動数合否:{0}'.format(log[f_axis>f_axis_req]))
    print('縦振動数安全率:{0}\n'.format(f_axis/f_axis_req))
    
    print('=================================')    
    f_rad = get_f_rad(material,M,L,r,t)
    f_rad_req = required[3]
    print('横振動数合否:{0}'.format(log[f_rad>f_rad_req]))
    print('横振動数安全率:{0}\n'.format(f_rad/f_rad_req))